In [ ]:
# -----------------------Export parameters at certain iterations--------
import sqlite3
import pandas as pd

# Define database file and output file
db_file = "/Users/......../pso_r_7.db"
txt_file_comsol_filtered = "comsol_params.txt"

# Connect to the database
conn = sqlite3.connect(db_file)

# Query to get available iterations
query_iterations = "SELECT DISTINCT iteration FROM particles ORDER BY iteration;"
df_iterations = pd.read_sql_query(query_iterations, conn)

# Print available iterations
print(df_iterations)

# Ask user to select iterations
selected_iterations = input("Enter the iterations you want to include (comma-separated): ")
selected_iterations = [int(x.strip()) for x in selected_iterations.split(",")]

# Query to get the best non-Inf PPI per selected iteration
query = f"""
SELECT loadb, Es, rhos, nus, cs, phis, epss, thetass, hks, H1, H2, x6, e_plst, sigma_1
FROM particles
WHERE ppi < 1e308
AND iteration IN ({','.join(map(str, selected_iterations))})
GROUP BY iteration
ORDER BY iteration;
"""

# Fetch data into a DataFrame
df_filtered = pd.read_sql_query(query, conn)

# Close the connection
conn.close()

# Define parameter names and units
parameter_units = {
    "loadb": "[Pa]",
    "Es": "[Pa]",
    "rhos": "[]",
    "nus": "[]",
    "cs": "[kPa]",
    "phis": "[rad]",
    "epss": "[]",
    "thetass": "[]",
    "hks": "[m/s]",
    "H1": "[m]",
    "H2": "[m]",
    "x6": "[]"
}

# Write to text file in the required format
with open(txt_file_comsol_filtered, "w") as f:
    for param in df_filtered.columns:
        values = ",".join(map(str, df_filtered[param].tolist()))
        unit = parameter_units.get(param, "[]")  # Default to empty brackets if unit is missing
        f.write(f"{param} {values} {unit}\n")

print(f"Exported COMSOL-compatible parameter sets for selected iterations to {txt_file_comsol_filtered}")
